# ALeRCE classes

https://github.com/ZwickyTransientFacility/ztf-avro-alert

1. **AGN:** Active Galactic Nuclei
1. **Blazar:** Blazar
1. **CV/Nova:** Cataclysmic Variable Star/Nova
1. **Ceph:** Cepheid Variable Star
1. **DSCT:** Delta Scuti Star
1. **EA:** Eclipsing Algol
1. **EB/EW:** Eclipsing Binaries/Eclipsing W Ursa Majoris
1. **LPV:** Long Period Variable
1. **Periodic-Other:** Periodic-Other
1. **QSO:** Quasi-Stellar Object
1. **RRL:** RRLyrae Variable Star
1. **RSCVn:** RS Canum Venaticorum
1. **SLSN:** Super Luminous Supernova
1. **SNII:** Supernova II
1. **SNIIb:** Supernova IIb
1. **SNIIn:** Supernova IIn
1. **SNIa:** Supernova Ia
1. **SNIbc:** Supernova Ibc
1. **TDE:** Tidal disruption event (to remove)
1. **YSO:** Young Stellar Object
1. **ZZ:** ZZ Ceti Stars (to remove)

In [ ]:
import sys
sys.path.append('../../')

In [ ]:
%load_ext autoreload
%autoreload 2
from mismatch.alerce_utils import process_df_labels, process_df_detections, keep_only_valid_objs
import numpy as np
import pandas as pd

load_rootdir = '../../../../tesis/surveys_data'
survey_name = 'alerceZTFv7.1'
df_index_names = {
    'oid':'oid', # object id
    'oid_det':'index', # object id
    'label':'classALeRCE', # object class name
    'ra':'ra',
    'dec':'dec',
    'band':'fid', # band
    'obs_day':'mjd', # days
    'obs':'magpsf_corr', # observations
    'obs_error':'sigmapsf_corr', # observation errors
}

### load files and processing
features_df = pd.read_parquet(f'{load_rootdir}/{survey_name}/storage/ztf_workspace/historic_data_20200916')
features_df = features_df.set_index(['index'])
features_df, det_objs = process_df_detections(features_df, df_index_names['oid_det'], df_index_names['oid'])
print(f'features_df - columns: {list(features_df.columns)} - id: {features_df.index.name}')

labels_df = pd.read_csv(f'{load_rootdir}/{survey_name}/dfcrossmatches_prioritized_v7.0.1.csv')
labels_df, label_objs = process_df_labels(labels_df, df_index_names['oid'], det_objs)
print(f'labels - columns: {list(labels_df.columns)} - id: {labels_df.index.name}')

### filter
#valid_objs = sorted(list(set(det_objs) & set(label_objs)))
#labels_df = keep_only_valid_objs(labels_df, valid_objs)
#features_df = keep_only_valid_objs(features_df, valid_objs)

### print info
classes = sorted(set(labels_df[df_index_names['label']].values))
print('classes:', classes)

In [ ]:
%load_ext autoreload
%autoreload 2
from mismatch.level_bars import LevelBar

classes, counts = np.unique(labels_df[df_index_names['label']].values, return_counts=True)
population_cdict = {c:counts[kc] for kc,c in enumerate(classes)}
print(LevelBar(population_cdict, ncols=60))

In [ ]:
print('features_df:', features_df.info())
features_df[:10]

In [ ]:
print('labels_df:', labels_df.info())
labels_df[:10]

In [ ]:
%load_ext autoreload
%autoreload 2
from mismatch.alerce_utils import get_train_test_split

train_features_df, test_features_df = get_train_test_split(features_df, labels_df, df_index_names['oid'])

# prints

In [ ]:
print(train_features_df.info())
train_features_df[:10]

In [ ]:
print(test_features_df.info())
test_features_df[:10]

In [ ]:
import pandas as pd

### save files
save_root_dir = f'../../data/{survey_name}'
labels_df.to_parquet(f'{save_root_dir}/labels.parquet')
train_features_df.to_parquet(f'{save_root_dir}/features_train.parquet')
test_features_df.to_parquet(f'{save_root_dir}/features_test.parquet')